# Вебинар 4. Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, find

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
recommender = MainRecommender(data_train)
recommender

In [4]:
display(recommender.get_similar_items_recommendation(2375))
recommender.get_similar_users_recommendation(2375)

[960733, 1092149, 1043590, 1038746, 1046545]

[1022920, 8020166, 946011, 903711, 917482]

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['bm25'] = result['user_id'].apply(lambda uid: recommender.get_recommendations(uid, N=5))
result.head(3)

,user_id,actual,bm25
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1082185, 5978656, 991580, 13157974, 938004]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1082185, 981760, 1098066, 826249, 1044078]"
2,5,"[913077, 1118028, 1386668]","[1082185, 916122, 1098066, 885406, 1001106]"


In [6]:
result['similar_items'] = result['user_id'].apply(lambda uid: recommender.get_similar_items_recommendation(uid, N=5))
result.head(3)

,user_id,actual,bm25,similar_items
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1082185, 5978656, 991580, 13157974, 938004]","[842762, 892264, 7441498, 1040807, 15926844]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1082185, 981760, 1098066, 826249, 1044078]","[10250632, 1092026, 910032, 892008, 951590]"
2,5,"[913077, 1118028, 1386668]","[1082185, 916122, 1098066, 885406, 1001106]","[6773188, 10355657, 1058997, 5995589, 930917]"


In [7]:
result['similar_users'] = result['user_id'].apply(lambda uid: recommender.get_similar_users_recommendation(uid, N=5))
result.head(3)

,user_id,actual,bm25,similar_items,similar_users
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1082185, 5978656, 991580, 13157974, 938004]","[842762, 892264, 7441498, 1040807, 15926844]","[1046219, 13157845, 1092295, 825160, 6513825]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1082185, 981760, 1098066, 826249, 1044078]","[10250632, 1092026, 910032, 892008, 951590]","[1076206, 884643, 902797, 7025260, 906202]"
2,5,"[913077, 1118028, 1386668]","[1082185, 916122, 1098066, 885406, 1001106]","[6773188, 10355657, 1058997, 5995589, 930917]","[998519, 1055548, 872826, 1089393, 1019146]"


In [8]:
for model_name in ['bm25', 'similar_items', 'similar_users']:
    mean_precision_at_k = result.apply(lambda row: precision_at_k(row[model_name], row['actual']), axis=1).mean()
    print(f'Model: {model_name}, Mean precision@k = {mean_precision_at_k}')

Model: bm25, Mean precision@k = 0.18925163234555267
Model: similar_items, Mean precision@k = 0.046308387744852145
Model: similar_users, Mean precision@k = 0.0025113008538422913


## Выводы:
* Оба подхода "топ-N товаров, среди купленных похожими юзерами" и "товары, похожие на топ-N купленных юзером товаров" оказались существенно хуже предсказания базовой модели (ALS + BM25). 